In [1]:
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt

/usr/local/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [28]:
home = '/Volumes/group/active/BABIES/BABIES_diffusion/subjsDir'
newborn = home + '/Newborn'
sixmonth = home + '/6-month'
out = '/Volumes/group/active/BABIES/BABIES_diffusion/check_motion/QC_plots_1.2019'
timepoint = newborn

In [29]:
sublist = [] #Abbreviated subject name: e.g. 125
ids = [] #Full subject name: e.g. 125-C-T1
def get_subs(timepoint):
    subpaths = glob.glob(timepoint + '/*-*-*')
    for sub in subpaths:
        sub1 = sub.replace(timepoint+'/', '')
        ids.append(sub1)
        sub2 = sub1.replace('-C-T1', '').replace('-L-T2', '').replace('-L-T3', '')
        sublist.append(sub2)
    return print('All subs extracted for timepoint')
get_subs(timepoint)  

All subs extracted for timepoint


In [ ]:
for sub in ids:
    try:
        eddymov = pd.read_csv(timepoint + '/{}/DTI_unwarped_eddy.eddy_movement_rms'.format(sub), delimiter = '  ', header=None)
        edmo = eddymov.rename(columns={0:'RMS Movement Relative to Volume 1 (Absolute)', 1: 'RMS Movement Relative to Previous Volume (Relative)'})
        x = lambda x: True if x % 2 >= 0 else False 
        edmo['Absolute Outlier'] = 
        plt.figure()
        edmo.plot()
        axes = plt.gca()
        axes.set_ylim([0,5])
        plt.title('{} RMS Movement (from FSL Eddy)'.format(sub), fontsize=16)
        plt.savefig(out+'/{}_RMS_Movement_Plot.png'.format(sub))
        plt.clf()
    except:
        print('Data not found for {}'.format(sub))

In [30]:
def get_true(value):
    if value > 1.999:
        return 1
    elif value < 1.999:
        return 0
    else:
        return 1

In [31]:
datadf = pd.DataFrame(columns = ['SubID', 'Volumes with Absolute Outliers', 'Volumes with Relative Outliers'])
for sub in ids:
    try:
        eddymov = pd.read_csv(timepoint + '/{}/DTI_unwarped_eddy.eddy_movement_rms'.format(sub), delimiter = '  ', header=None)
        edmo = eddymov.rename(columns={0:'RMS Movement Relative to Volume 1 (Absolute)', 1: 'RMS Movement Relative to Previous Volume (Relative)'})
        edmo['Absolute Outlier'] = edmo['RMS Movement Relative to Volume 1 (Absolute)'].apply(get_true)
        edmo['Relative Outlier'] = edmo['RMS Movement Relative to Previous Volume (Relative)'].apply(get_true)
        edmo.index.rename('Volumes', inplace = True)
        abs_outliers = edmo.index[edmo['Absolute Outlier'] == 1].tolist()
        rel_outliers = edmo.index[edmo['Relative Outlier'] == 1].tolist()
        datadf = datadf.append({'SubID': sub, 'Volumes with Absolute Outliers': abs_outliers, 'Volumes with Relative Outliers':rel_outliers}, ignore_index=True)
    except:
        print('Data not found for {}'.format(sub))


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


Data not found for 1042-L-T2
Data not found for 1011-L-T2
Data not found for File_QC_Report_2018-08-21.csv


In [33]:
if timepoint == sixmonth:
    datadf.to_csv(out+'/06month_RMS_Movement_AllSubs.csv')
elif timepoint == newborn:
    datadf.to_csv(out+'/0Newborn_RMS_Movement_AllSubs.csv')